In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 100
Λ = model_problem_spectrum(n,.6,1e3)
λmin = np.min(Λ)
λmax = np.max(Λ)

A = np.diag(Λ)

b = np.ones(n)
b /= np.linalg.norm(b)

f = lambda x: 1/x

fAb = np.diag(f(Λ))@b

In [4]:
k_max = 61
Q,(α,β) = lanczos_reorth(A,b,k_max,reorth=0)

err_FA = np.full(k_max,np.nan)
for k in range(k_max):
    err_FA[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAb - lanczos_FA(f,Q,α,β,k)))

In [5]:
k_max_ro = 24
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,b,k_max_ro,reorth=k_max_ro)

err_FA_ro = np.full(k_max_ro,np.nan)
for k in range(k_max_ro):
    err_FA_ro[k] = np.linalg.norm(np.diag(np.sqrt(Λ))@(fAb - lanczos_FA(f,Q_ro,α_ro,β_ro,k)))

In [6]:
err_κ = 2*np.exp(-2*np.arange(k_max)/np.sqrt(λmax/λmin))

In [7]:
err_Λ = np.full(k_max_ro,np.nan)
err_Λ[0] = 1
i=[1]
for k in range(k_max_ro-1):
    i_add = np.argmax(np.min(np.abs(Λ[i]-Λ[:,None]),axis=1))
    i = np.append(i,i_add)
    i = np.sort(i)
    i,p,ub,lb = remez_zero(f,Λ,k,i=i,tol=1e-10,max_iter=300)
    err_Λ[k+1] = ub

In [8]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(err_Λ,**line_styles['l4'],label='bound on $\Lambda$')
ax.plot(err_κ,**line_styles['l3'],label=r'bound on $\mathcal{I}$')
ax.plot(err_FA_ro/err_FA_ro[0],**line_styles['l2'],label='reorth')
ax.plot(err_FA/err_FA[0],**line_styles['l1'],label='no reorth')

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_ylabel('error norm')
ax.set_yscale('log')

plt.savefig('imgs/unif_vs_spec.pdf')
plt.close()